# Task 13.6: Agent Benchmarking & Evaluation Framework

**Module:** 13 - AI Agents & Agentic Systems  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand how to evaluate AI agent performance
- [ ] Build a comprehensive benchmarking framework
- [ ] Implement metrics for retrieval, generation, and tool use
- [ ] Create test suites for your agents
- [ ] Generate actionable performance reports

---

## 📚 Prerequisites

- Completed: Tasks 13.1-13.5
- Knowledge of: Testing, metrics, evaluation concepts

---

## 🌍 Real-World Context

**Why benchmark agents?**

Building an agent is just the beginning. You need to know:
- 🎯 **Accuracy**: Does it give correct answers?
- ⚡ **Speed**: Is it fast enough for your use case?
- 🔧 **Reliability**: Does it consistently work?
- 📊 **Improvement**: How do changes affect performance?

**Real scenarios:**
- 🏢 **Enterprise**: "Can we deploy this agent to 1000 users?"
- 🔬 **Research**: "Did our improvements actually help?"
- 💰 **Business**: "What's the ROI on this AI investment?"

---

## 🧒 ELI5: Agent Evaluation

> **Imagine you're grading a student's test...** 📝
>
> You don't just mark answers right or wrong. You look at:
> - Did they understand the question?
> - Was their reasoning correct?
> - Did they show their work?
> - How fast did they finish?
>
> **Agent evaluation is the same!**
> - **Retrieval**: Did it find the right information?
> - **Reasoning**: Did it use the right tools?
> - **Generation**: Was the answer helpful?
> - **Latency**: How long did it take?

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
import json
import time
import statistics
from datetime import datetime

# Add scripts directory with robust path resolution
# This handles cases where the notebook is run from different working directories
NOTEBOOK_DIR = Path(os.getcwd())

# Try multiple approaches to find the scripts directory
scripts_paths = [
    NOTEBOOK_DIR.parent / 'scripts',  # Standard case: running from notebooks/
    NOTEBOOK_DIR / 'scripts',          # Running from module root
    Path(__file__).parent.parent / 'scripts' if '__file__' in dir() else None,
]

scripts_found = False
for scripts_path in scripts_paths:
    if scripts_path and scripts_path.exists():
        sys.path.insert(0, str(scripts_path))
        scripts_found = True
        print(f"✅ Scripts directory found: {scripts_path}")
        break

if not scripts_found:
    print("⚠️ Scripts directory not found. Trying import anyway...")

# Import our benchmark utilities
try:
    from benchmark_utils import (
        TestCase, TestResult, BenchmarkResults, TestCategory, Difficulty,
        AgentEvaluator, keyword_match_score, f1_score, rouge_l_score,
        generate_report, save_results_json, load_test_cases_from_json
    )
    print("✅ Imports successful!")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("   Make sure you're running from the notebooks/ directory")
    print("   Or run: cd phase-3-advanced/module-13-ai-agents/notebooks")

In [ ]:
# Initialize LLM for testing
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3.1:8b",
    temperature=0.1,  # Low temperature for more consistent results
    base_url="http://localhost:11434"
)

print("LLM initialized for evaluation!")

---

## Part 2: Creating Test Cases

Good test cases are specific, measurable, and cover various scenarios.

In [ ]:
# Define test cases for our RAG system
rag_test_cases = [
    # Factual retrieval - Easy
    TestCase(
        id="rag_001",
        query="What is the memory capacity of DGX Spark?",
        expected_answer="128GB unified LPDDR5X memory",
        category=TestCategory.FACTUAL_RETRIEVAL,
        difficulty=Difficulty.EASY,
        keywords=["128GB", "unified", "memory", "LPDDR5X"]
    ),
    TestCase(
        id="rag_002",
        query="How many CUDA cores does the Blackwell GB10 have?",
        expected_answer="6,144 CUDA cores",
        category=TestCategory.FACTUAL_RETRIEVAL,
        difficulty=Difficulty.EASY,
        keywords=["6144", "CUDA", "cores"]
    ),
    
    # Multi-hop reasoning - Medium
    TestCase(
        id="rag_003",
        query="Can a 70B parameter model fit in DGX Spark's memory in FP16?",
        expected_answer="Yes, a 70B model in FP16 requires about 140GB, and DGX Spark has 128GB. With some optimization it can fit.",
        category=TestCategory.MULTI_HOP_REASONING,
        difficulty=Difficulty.MEDIUM,
        keywords=["70B", "FP16", "memory", "fit"]
    ),
    TestCase(
        id="rag_004",
        query="What precision formats does DGX Spark's Blackwell architecture support natively?",
        expected_answer="FP4, FP8, bfloat16",
        category=TestCategory.FACTUAL_RETRIEVAL,
        difficulty=Difficulty.MEDIUM,
        keywords=["FP4", "FP8", "bfloat16", "Blackwell"]
    ),
    
    # Complex reasoning - Hard
    TestCase(
        id="rag_005",
        query="Compare the advantages of LoRA vs full fine-tuning for a 70B model on DGX Spark.",
        expected_answer="LoRA allows fine-tuning with minimal additional parameters (0.1% of total), enabling 70B model fine-tuning on DGX Spark's 128GB memory. Full fine-tuning would require storing optimizer states for all parameters.",
        category=TestCategory.MULTI_HOP_REASONING,
        difficulty=Difficulty.HARD,
        keywords=["LoRA", "fine-tuning", "memory", "parameters"]
    ),
]

print(f"Created {len(rag_test_cases)} RAG test cases")
for tc in rag_test_cases:
    print(f"  - {tc.id}: {tc.category.value} ({tc.difficulty.value})")

In [ ]:
# Define test cases for tool use
tool_test_cases = [
    TestCase(
        id="tool_001",
        query="Calculate 15% tip on a $127.50 restaurant bill.",
        expected_answer="$19.13",
        category=TestCategory.CALCULATION,
        difficulty=Difficulty.EASY,
        keywords=["19.13", "19.125"],
        requires_tool="calculator"
    ),
    TestCase(
        id="tool_002",
        query="What is the square root of 256 multiplied by 4?",
        expected_answer="64",
        category=TestCategory.CALCULATION,
        difficulty=Difficulty.EASY,
        keywords=["64"],
        requires_tool="calculator"
    ),
    TestCase(
        id="tool_003",
        query="Calculate how many 7B parameter models (at 2 bytes per parameter) can fit in 128GB.",
        expected_answer="About 9 models (128GB / 14GB per model)",
        category=TestCategory.CALCULATION,
        difficulty=Difficulty.MEDIUM,
        keywords=["9", "models"],
        requires_tool="calculator"
    ),
]

print(f"\nCreated {len(tool_test_cases)} tool use test cases")

---

## Part 3: Understanding Evaluation Metrics

In [ ]:
# Let's understand each metric

# Sample response and expected answer
expected = "The DGX Spark has 128GB of unified LPDDR5X memory."
response1 = "DGX Spark features 128GB unified memory using LPDDR5X technology."  # Good
response2 = "The system has a lot of memory."  # Vague
response3 = "DGX Spark has 64GB of DDR5 memory."  # Wrong

print("METRIC COMPARISON")
print("="*60)
print(f"Expected: {expected}")
print("\nResponses:")
print(f"  1 (Good): {response1}")
print(f"  2 (Vague): {response2}")
print(f"  3 (Wrong): {response3}")

In [ ]:
# Metric 1: Keyword Match
keywords = ["128GB", "unified", "LPDDR5X", "memory"]

print("\n1. KEYWORD MATCH SCORE")
print("   Measures: What % of expected keywords appear in response")
print("-"*40)

for name, response in [("Good", response1), ("Vague", response2), ("Wrong", response3)]:
    score = keyword_match_score(response, keywords)
    print(f"   {name}: {score:.2%}")

In [ ]:
# Metric 2: F1 Score (Token Overlap)
print("\n2. F1 SCORE (Token Overlap)")
print("   Measures: Balance of precision and recall on words")
print("-"*40)

expected_tokens = expected.lower().split()

for name, response in [("Good", response1), ("Vague", response2), ("Wrong", response3)]:
    response_tokens = response.lower().split()
    score = f1_score(response_tokens, expected_tokens)
    print(f"   {name}: {score:.2%}")

In [ ]:
# Metric 3: ROUGE-L (Longest Common Subsequence)
print("\n3. ROUGE-L SCORE")
print("   Measures: Longest common subsequence similarity")
print("-"*40)

for name, response in [("Good", response1), ("Vague", response2), ("Wrong", response3)]:
    score = rouge_l_score(response, expected)
    print(f"   {name}: {score:.2%}")

---

## Part 4: Building the Evaluation Framework

In [ ]:
# Create a simple agent for testing
def create_simple_rag_agent(llm):
    """Create a simple RAG agent for testing."""
    # Simulated knowledge base
    knowledge = """
    DGX Spark is NVIDIA's desktop AI supercomputer featuring:
    - 128GB unified LPDDR5X memory
    - Blackwell GB10 Superchip with 6,144 CUDA cores
    - 192 5th-generation Tensor Cores
    - Native support for FP4, FP8, and bfloat16 precision
    - Up to 1 PFLOP FP4 compute
    
    LoRA (Low-Rank Adaptation) enables efficient fine-tuning by:
    - Training only 0.1% of total parameters
    - Keeping base model weights frozen
    - Using rank decomposition matrices
    """
    
    def agent_func(query: str) -> str:
        prompt = f"""Based on this knowledge:
{knowledge}

Answer the question concisely:
{query}

Answer:"""
        return llm.invoke(prompt)
    
    return agent_func

# Create the agent
test_agent = create_simple_rag_agent(llm)
print("Test agent created!")

In [ ]:
# Test the agent
test_query = "What is the memory capacity of DGX Spark?"
response = test_agent(test_query)

print(f"Query: {test_query}")
print(f"Response: {response}")

In [ ]:
# Create the evaluator
evaluator = AgentEvaluator(
    agent_func=test_agent,
    embedding_model=None,  # Skip semantic similarity for speed
    verbose=True
)

print("Evaluator created!")

---

## Part 5: Running the Benchmark

In [ ]:
# Run benchmark on RAG test cases
print("\n" + "="*60)
print("RUNNING RAG BENCHMARK")
print("="*60 + "\n")

rag_results = evaluator.run_benchmark(
    test_cases=rag_test_cases,
    name="RAG Pipeline Evaluation"
)

In [ ]:
# Generate and display the report
report = generate_report(rag_results)
print(report)

In [ ]:
# Detailed look at individual results
print("\nDETAILED RESULTS")
print("="*60)

for result in rag_results.results:
    print(f"\nTest: {result.test_case.id}")
    print(f"Query: {result.test_case.query}")
    print(f"Expected: {result.test_case.expected_answer[:100]}...")
    print(f"Got: {result.agent_response[:100]}...")
    print(f"Score: {result.score:.2%} | Latency: {result.latency_ms:.0f}ms")
    print(f"Metrics: {result.metrics}")

---

## Part 6: Comparing Agents

In [ ]:
# Create a second agent with different settings
def create_detailed_agent(llm):
    """Agent that gives more detailed responses."""
    knowledge = """
    DGX Spark is NVIDIA's desktop AI supercomputer featuring:
    - 128GB unified LPDDR5X memory
    - Blackwell GB10 Superchip with 6,144 CUDA cores
    - 192 5th-generation Tensor Cores
    """
    
    def agent_func(query: str) -> str:
        prompt = f"""Based on this knowledge:
{knowledge}

Provide a detailed, comprehensive answer to:
{query}

Include specific numbers and technical details.

Answer:"""
        return llm.invoke(prompt)
    
    return agent_func

detailed_agent = create_detailed_agent(llm)
print("Detailed agent created!")

In [ ]:
# Evaluate both agents
from benchmark_utils import compare_agents

# Evaluate simple agent
simple_evaluator = AgentEvaluator(test_agent, verbose=False)
simple_results = simple_evaluator.run_benchmark(rag_test_cases, "Simple Agent")

# Evaluate detailed agent
detailed_evaluator = AgentEvaluator(detailed_agent, verbose=False)
detailed_results = detailed_evaluator.run_benchmark(rag_test_cases, "Detailed Agent")

# Compare
comparison = compare_agents(
    results_list=[simple_results, detailed_results],
    agent_names=["Simple Agent", "Detailed Agent"]
)

print(comparison)

---

## Part 7: Saving and Loading Results

In [ ]:
# Save results to JSON
output_dir = Path.cwd().parent / "data" / "benchmark_results"
output_dir.mkdir(parents=True, exist_ok=True)

results_file = output_dir / f"rag_benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
save_results_json(rag_results, str(results_file))

print(f"Results saved to: {results_file}")

In [ ]:
# Load and view saved results
with open(results_file, 'r') as f:
    loaded_data = json.load(f)

print("Loaded benchmark results:")
print(f"  Name: {loaded_data['name']}")
print(f"  Overall Score: {loaded_data['overall_score']:.2%}")
print(f"  Test Count: {len(loaded_data['results'])}")

---

## Part 8: Building a Test Suite

In [ ]:
class AgentTestSuite:
    """A comprehensive test suite for AI agents."""
    
    def __init__(self, agent_func: Callable, name: str):
        self.agent_func = agent_func
        self.name = name
        self.test_cases: Dict[str, List[TestCase]] = {
            'retrieval': [],
            'reasoning': [],
            'tool_use': [],
        }
        self.results: Dict[str, BenchmarkResults] = {}
    
    def add_test_cases(self, category: str, cases: List[TestCase]):
        """Add test cases to a category."""
        if category not in self.test_cases:
            self.test_cases[category] = []
        self.test_cases[category].extend(cases)
    
    def run_all(self, verbose: bool = True) -> Dict[str, BenchmarkResults]:
        """Run all test categories."""
        evaluator = AgentEvaluator(self.agent_func, verbose=verbose)
        
        for category, cases in self.test_cases.items():
            if cases:
                if verbose:
                    print(f"\n{'='*60}")
                    print(f"Running {category.upper()} tests...")
                    print('='*60)
                
                self.results[category] = evaluator.run_benchmark(
                    cases, f"{self.name} - {category}"
                )
        
        return self.results
    
    def get_summary(self) -> str:
        """Get a summary of all test results."""
        lines = [
            "="*60,
            f"TEST SUITE SUMMARY: {self.name}",
            "="*60,
            ""
        ]
        
        total_tests = 0
        total_passed = 0
        
        for category, result in self.results.items():
            passed = sum(1 for r in result.results if r.passed)
            total = len(result.results)
            total_tests += total
            total_passed += passed
            
            lines.append(f"{category.capitalize()}:")
            lines.append(f"  Score: {result.overall_score:.1%}")
            lines.append(f"  Passed: {passed}/{total}")
            lines.append(f"  Avg Latency: {result.timing_stats['mean_latency_ms']:.0f}ms")
            lines.append("")
        
        lines.append("-"*40)
        lines.append(f"OVERALL: {total_passed}/{total_tests} tests passed")
        lines.append("="*60)
        
        return "\n".join(lines)

print("AgentTestSuite class defined!")

In [ ]:
# Create and run a test suite
suite = AgentTestSuite(test_agent, "RAG Agent v1.0")

# Add test cases
suite.add_test_cases('retrieval', [
    tc for tc in rag_test_cases if tc.category == TestCategory.FACTUAL_RETRIEVAL
])
suite.add_test_cases('reasoning', [
    tc for tc in rag_test_cases if tc.category == TestCategory.MULTI_HOP_REASONING
])

# Run all tests
all_results = suite.run_all(verbose=True)

# Get summary
print("\n" + suite.get_summary())

---

## ⚠️ Common Mistakes

### Mistake 1: Only Testing Happy Paths

In [ ]:
# ❌ Wrong: Only test cases where the answer is in the knowledge
# good_case = TestCase(query="What is DGX Spark?", ...)

# ✅ Right: Also test edge cases
edge_cases = [
    # Out of domain
    TestCase(
        id="edge_001",
        query="What is the capital of France?",
        expected_answer="I don't have information about that",
        category=TestCategory.FACTUAL_RETRIEVAL,
        difficulty=Difficulty.EASY,
        keywords=["don't", "not", "outside"]
    ),
    # Ambiguous
    TestCase(
        id="edge_002",
        query="Is it good?",
        expected_answer="I need more context to answer that question",
        category=TestCategory.FACTUAL_RETRIEVAL,
        difficulty=Difficulty.MEDIUM,
        keywords=["context", "clarify", "specific"]
    ),
]

print("Always test edge cases and failure modes!")

### Mistake 2: Ignoring Latency

In [ ]:
# ❌ Wrong: Only measuring accuracy
# score = accuracy_only(response, expected)

# ✅ Right: Track latency as a key metric
def evaluate_with_latency(agent_func, query):
    start = time.time()
    response = agent_func(query)
    latency = (time.time() - start) * 1000  # ms
    
    return {
        'response': response,
        'latency_ms': latency,
        'acceptable': latency < 5000  # 5 second threshold
    }

print("Latency matters for user experience!")

---

## 🎉 Checkpoint

You've learned:
- ✅ Why agent evaluation is crucial
- ✅ Different metrics: keyword match, F1, ROUGE-L
- ✅ Building test cases for various scenarios
- ✅ Creating a benchmarking framework
- ✅ Comparing different agent configurations
- ✅ Saving and analyzing results

---

## 🚀 Challenge (Optional)

Build a continuous evaluation pipeline that:
1. Runs automatically on code changes
2. Tracks metrics over time
3. Alerts when performance degrades
4. Generates trend reports

---

## 📖 Further Reading

- [RAGAS - RAG Assessment](https://docs.ragas.io/)
- [LangSmith Evaluation](https://docs.smith.langchain.com/)
- [Eleuther AI LM Eval Harness](https://github.com/EleutherAI/lm-evaluation-harness)

---

## 🧹 Cleanup

In [ ]:
# Comprehensive cleanup for DGX Spark
import gc

# Clear GPU memory if available
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"✅ GPU memory cleared ({allocated:.2f} GB still allocated)")
except ImportError:
    pass

# Python garbage collection
gc.collect()
print("✅ Cleanup complete!")

---

## 🎓 Summary

In this notebook, you built a comprehensive agent evaluation framework:

1. **Test Cases**: Structured tests with expected answers
2. **Metrics**: Keyword match, F1, ROUGE-L, latency
3. **Evaluator**: Automated testing and scoring
4. **Test Suite**: Organized test categories
5. **Reports**: Actionable performance summaries

**Key takeaways:**
- Always measure before and after changes
- Test edge cases, not just happy paths
- Track latency alongside accuracy
- Save results for historical comparison

**Congratulations! You've completed Module 13: AI Agents & Agentic Systems!**